In [2]:
import os
import sys
import geopandas as gpd
import torch
sys.path.append("../scripts/")
from asm_datamodules import *
from asm_losses import *

In [21]:
%load_ext autoreload
%autoreload 2

In [2]:
path="/n/home07/kayan/asm/data/filtered_labels.geojson"
label_df = gpd.read_file(path)

In [3]:
label_df.head()

,lon,lat,unique_id,country,province,input_id,sample_type,mine_type,confidence,label,area_km2,proportion_inspected,proportion_mining,assumed_label,2_cell_buffer,3_cell_buffer,4_cell_buffer,5_cell_buffer,geometry
0,-12.885,9.155,lat_9--155__lon_-12--885,SLE,Northern,SLE-1279CL,IPIS,artisanal,3.0,1.0,0.000656,0.523869,0.000523,positive,True,True,True,True,"POLYGON ((-12.88452 9.15891, -12.88457 9.15875..."
1,-12.875,9.175,lat_9--175__lon_-12--875,SLE,Northern,SLE-184CL,IPIS,artisanal,4.0,1.0,0.000587,0.350198,0.000468,positive,True,True,True,True,"MULTIPOLYGON (((-12.87024 9.17627, -12.87035 9..."
2,-12.865,9.175,lat_9--175__lon_-12--865,SLE,Northern,SLE-184CL,IPIS,artisanal,4.0,1.0,0.003670,0.384139,0.002924,positive,True,True,True,True,"POLYGON ((-12.86941 9.17610, -12.86930 9.17596..."
3,-12.855,9.125,lat_9--125__lon_-12--855,SLE,Northern,negatives-1428,CLU,artisanal,3.0,1.0,0.001271,0.784137,0.001013,negative,False,True,True,True,"POLYGON ((-12.85544 9.12746, -12.85553 9.12733..."
4,-12.835,8.925,lat_8--925__lon_-12--835,SLE,Northern,negatives-638,CLU,artisanal,3.0,1.0,0.001501,0.784137,0.001197,negative,False,False,False,True,"POLYGON ((-12.83892 8.92752, -12.83883 8.92757..."


In [7]:
# device configuration
device, num_devices = ("cuda", torch.cuda.device_count()) if torch.cuda.is_available() else ("cpu", len(os.sched_getaffinity(0)))
workers = len(os.sched_getaffinity(0))

In [24]:
# datamodule configuration
num_workers = workers # set to maximum number of available CPUs
root = "/n/holyscratch01/tambe_lab/kayan/karena/" # root for data files
split_path = "../data/split_regular" # one of the randomly generated splits

# create and set up datamodule
datamodule = ASMDataModule(batch_size=64, 
                           num_workers=num_workers, 
                           root=root,
                           transforms=min_max_transform,
                           augment=False,
                           split_path=split_path)

In [25]:
datamodule.setup("fit")
train_dataloader = datamodule.train_dataloader()

In [26]:
for batch in train_dataloader:
    print(batch)
    break

{'image': tensor([[[[0.3550, 0.4231, 0.4639,  ..., 0.4696, 0.5120, 0.3844],
          [0.4288, 0.4425, 0.5020,  ..., 0.4090, 0.4603, 0.3475],
          [0.4730, 0.4698, 0.4965,  ..., 0.3801, 0.3858, 0.3241],
          ...,
          [0.5184, 0.5656, 0.5381,  ..., 0.1840, 0.2068, 0.2238],
          [0.5014, 0.5553, 0.5396,  ..., 0.1415, 0.1512, 0.1697],
          [0.4560, 0.5099, 0.4719,  ..., 0.1326, 0.1215, 0.1498]],

         [[0.3959, 0.4821, 0.5056,  ..., 0.5088, 0.5063, 0.3993],
          [0.4464, 0.4990, 0.5408,  ..., 0.4415, 0.4674, 0.3785],
          [0.4566, 0.4972, 0.5344,  ..., 0.3915, 0.4051, 0.3352],
          ...,
          [0.4868, 0.5491, 0.5539,  ..., 0.1836, 0.2190, 0.2511],
          [0.4529, 0.5343, 0.5630,  ..., 0.1685, 0.1743, 0.2029],
          [0.4232, 0.4916, 0.5044,  ..., 0.1675, 0.1464, 0.1672]],

         [[0.4025, 0.4748, 0.4855,  ..., 0.4740, 0.5014, 0.3568],
          [0.4892, 0.4923, 0.5381,  ..., 0.4572, 0.4794, 0.3568],
          [0.5086, 0.5269, 0.553

In [22]:
# Define the batch size
batch_size = 10
num_classes = 2
height = 256
width = 256

# Generate random 2D data for input predictions (e.g., model outputs)
y_pred = torch.randn(batch_size, num_classes, height, width)

# Generate random 2D data for target labels
y_true = torch.randint(0, num_classes, (batch_size, height, width))

# Generate random confidence scores
confidence_scores = torch.rand(batch_size)

In [43]:
loss_fn = ConfidenceLoss()

In [44]:
loss_fn(y_pred,y_true,confidence_scores)

tensor(0.0526)